## Produce good and bad images for training and testing
Using this notebook, you can quickly produce good and bad datasets. Basic interactive console to start building datasets. Producing a variety of results by changing clothing, camera angle, body position, lighting, etc. It is not recommended to choose a large batch size. Try it out with a small batch size and adjust the `frames_to_hold` variable up for faster hardware and down for slower hardware. This is both to avoid false pauses in motion due as well as to avoid fatigue from holding a squat for too long. By default, this notebook will speak "ok", everytime it captures. Setting `use_speech = False` will disable this feature. DISCLAIMER: Doing bad squats on purpose is bad for your health. We do not recommend doing any exercise incorrectly.
<br><br>
The program will ask you for input of 'g' or 'b' for good or bad, or 'q'  to quit the loop and complete the cells execution, saving the images.  You may use the same loop to do both good and bad until you are done and choose 'q', or provide keyboard_interrupt.

In [ ]:
from IPython.display import clear_output, Image, display, HTML, Audio, Javascript as js
import cv2
import numpy as np
from skimage import metrics
from datetime import datetime

In [ ]:
# Disable speech notification below:
use_speech = True
def speak(display_handle,text):
    
    text = text.replace("'",r"\'")
    display_handle.update(js('''
    if(window.speechSynthesis) {{
    var synth = window.speechSynthesis;
    synth.speak(new window.SpeechSynthesisUtterance('{text}'));
    }}'''.format(text=text)))
#     clear_output(False)

In [ ]:
video = cv2.VideoCapture(0)

prev_frame = None
ssim = 0.0
count=0
image_array = []

display_handle=display(None, display_id=True)
sentinel = True
save_dir = ""
good_dir = "./good/"
bad_dir = "./bad/"
frames_to_hold = 20

while sentinel:
    image_array = []
    squat_type = input("choose (g)ood, (b)ad, (q)uit.")
    
    
    if squat_type == "q":
        break
    elif squat_type == "g":
        save_dir = good_dir
    elif squat_type == "b":
        save_dir = bad_dir
    else:
        print("bad choice.")
        continue
    batch_size = int(input("choose batch size: "))
    try:
        while batch_size > 0:
            

            _, frame = video.read()

            try:
                ssim = metrics.structural_similarity(prev_frame,frame,multichannel=True)
            except:
                pass
            if ssim > 0.9:
                count += 1
            if count > frames_to_hold:
                if use_speech:
                    speak(display_handle,"ok")
                image_array.append(frame)
                batch_size -= 1
                count = 0
            prev_frame = frame
            display_handle.update(Image(data=cv2.imencode('.jpeg', frame)[1]))

    except KeyboardInterrupt:
        pass
    finally:
        video.release()
        display_handle.update(None)
    now = datetime.now()
    now = now.strftime("%Y%m%d%H%M%S")
    for i in range(len(image_array)):
        cv2.imwrite(save_dir + "img%ddate%s.jpg"%(i,now),image_array[i])
        

#### Sources:
- https://stackoverflow.com/questions/27882255/is-it-possible-to-display-an-opencv-video-inside-the-ipython-jupyter-notebook
- https://stackoverflow.com/questions/58861577/differences-between-pil-image-open-and-cv2-imdecode
- https://www.geeksforgeeks.org/python-opencv-cv2-imread-method/
- https://stackoverflow.com/questions/44328645/interpreting-cv2-imencode-result
- https://docs.opencv.org/3.4/d8/d6a/group__imgcodecs__flags.html
- https://stackoverflow.com/questions/19098104/python-opencv2-cv2-wrapper-to-get-image-size
- https://mindtrove.info/jupyter-tidbit-run-and-say-done